In [1]:
# LIBRAIRES
import numpy as np
import pandas as pd
import random
import pickle

import string

from matplotlib import pyplot as plt 

In [2]:
# METHODS
def swap_vals(dirty_df, n):
    mods = pd.DataFrame()
    for i in range(n):
        col_i = random.randint(0,dirty_df.shape[1]-1)
        col = dirty_df.columns[col_i]
        row_1 = random.randint(0,dirty_df.shape[0]-1)
        row_2 = random.randint(0,dirty_df.shape[0]-1)
        # val = dirty_df.iloc[row][col]
        val_1 = dirty_df.iat[row_1,col_i]
        val_2 = dirty_df.iat[row_2,col_i]
    
        dirty_df.iat[row_1,col_i] = val_2
        dirty_df.iat[row_2,col_i] = val_1
    
        mod = {'col': col, 'row 1': row_1, 'val 1': val_1,'row 2': row_2,'val 2':val_2}
        # mods = mods.append(mod, ignore_index = True)
        mods = pd.concat([mods, pd.DataFrame([mod])], ignore_index=True)
    return(dirty_df,mods)

def rand_mod(dirty_df, n):
    mods = pd.DataFrame()
    for i in range(n):
        col_i = random.randint(0,dirty_df.shape[1]-1)
        col = dirty_df.columns[col_i]
        row_1 = random.randint(0,dirty_df.shape[0]-1)
        row_2 = random.randint(0,dirty_df.shape[0]-1)
        # val = dirty_df.iloc[row][col]
        val_1 = dirty_df.iat[row_1,col_i]
        val_2 = dirty_df.iat[row_2,col_i]
    
        dirty_df.iat[row_1,col_i] = val_2
        dirty_df.iat[row_2,col_i] = val_1
    
        mod = {'col': col, 'row 1': row_1, 'val 1': val_1,'row 2': row_2,'val 2':val_2}
        # mods = mods.append(mod, ignore_index = True)
        mods = pd.concat([mods, pd.DataFrame([mod])], ignore_index=True)

    return(dirty_df,mods)

In [3]:
# GET DATA
df = pd.read_csv('datasets/compas/compas-scores.csv')
df.shape

(11757, 47)

In [4]:
df.head()

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,vr_offense_date,vr_charge_desc,v_type_of_assessment,v_decile_score,v_score_text,v_screening_date,type_of_assessment,decile_score.1,score_text,screening_date
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,NaN,NaN,Risk of Violence,1,Low,2013-08-14,Risk of Recidivism,1,Low,2013-08-14
1,2,michael ryan,michael,ryan,2014-12-31,Male,1985-02-06,31,25 - 45,Caucasian,...,NaN,NaN,Risk of Violence,2,Low,2014-12-31,Risk of Recidivism,5,Medium,2014-12-31
2,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,2013-07-05,Felony Battery (Dom Strang),Risk of Violence,1,Low,2013-01-27,Risk of Recidivism,3,Low,2013-01-27
3,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,...,NaN,NaN,Risk of Violence,3,Low,2013-04-14,Risk of Recidivism,4,Low,2013-04-14
4,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,...,NaN,NaN,Risk of Violence,6,Medium,2013-01-13,Risk of Recidivism,8,High,2013-01-13


In [5]:
df['v_score_text'].nunique()
# df['v_score_text']

3

In [6]:
# CLEAN SOME DATATYPES
df["compas_screening_date"] = df["compas_screening_date"].astype("datetime64[ns]")
df["dob"] = df["dob"].astype("datetime64[ns]")
df["c_jail_in"] = df["c_jail_in"].astype("datetime64[ns]")
df["c_jail_out"] = df["c_jail_out"].astype("datetime64[ns]")
df["c_offense_date"] = df["c_offense_date"].astype("datetime64[ns]")
df["r_offense_date"] = df["r_offense_date"].astype("datetime64[ns]")
df["c_arrest_date"] = df["c_arrest_date"].astype("datetime64[ns]")
df["vr_offense_date"] = df["vr_offense_date"].astype("datetime64[ns]")
df["v_screening_date"] = df["v_screening_date"].astype("datetime64[ns]")
df["r_jail_in"] = df["r_jail_in"].astype("datetime64[ns]")
df["r_jail_out"] = df["r_jail_out"].astype("datetime64[ns]")
df["screening_date"] = df["screening_date"].astype("datetime64[ns]")


# drop some columns
cols = ['id','name','first','last','age_cat','c_case_number','r_case_number','vr_case_number']
df = df.drop(columns=cols) # remove label

df.dtypes


compas_screening_date      datetime64[ns]
sex                                object
dob                        datetime64[ns]
age                                 int64
race                               object
juv_fel_count                       int64
decile_score                        int64
juv_misd_count                      int64
juv_other_count                     int64
priors_count                        int64
days_b_screening_arrest           float64
c_jail_in                  datetime64[ns]
c_jail_out                 datetime64[ns]
c_offense_date             datetime64[ns]
c_arrest_date              datetime64[ns]
c_days_from_compas                float64
c_charge_degree                    object
c_charge_desc                      object
is_recid                            int64
num_r_cases                       float64
r_charge_degree                    object
r_days_from_arrest                float64
r_offense_date             datetime64[ns]
r_charge_desc                     

In [7]:
# one hot encode some cols
df = pd.get_dummies(data=df, columns=['sex'])
df = pd.get_dummies(data=df, columns=['c_charge_degree'])
df = pd.get_dummies(data=df, columns=['c_charge_desc'])
df = pd.get_dummies(data=df, columns=['r_charge_degree'])
df = pd.get_dummies(data=df, columns=['r_charge_desc'])
df = pd.get_dummies(data=df, columns=['vr_charge_degree'])
df = pd.get_dummies(data=df, columns=['vr_charge_desc'])
df = pd.get_dummies(data=df, columns=['v_type_of_assessment'])
df = pd.get_dummies(data=df, columns=['v_score_text'])
df = pd.get_dummies(data=df, columns=['type_of_assessment'])
df = pd.get_dummies(data=df, columns=['score_text'])

In [8]:
print(df.race.unique())
map = {
    'Caucasian': 0,
    'African-American': 1,
    'Hispanic': 1,
    'Asian': 1,
    'Native American': 1,
    'Other': 1
}

Y = df['is_recid'].values
P_val = df['race'].values
P = df['race'].map(map).values

df = pd.get_dummies(data=df, columns=['race']) # one hot encode race
df = df.drop(columns=['is_recid']) # remove label

['Other' 'Caucasian' 'African-American' 'Hispanic' 'Asian'
 'Native American']


In [9]:
df_dirty = df.copy()
df_dirty,swap_mods=swap_vals(df_dirty, 1000)
df_dirty,rand_mods=rand_mod(df_dirty, 10000)
# mods

In [10]:
# convert dates to integers
df["compas_screening_date"] = df["compas_screening_date"].dt.strftime("%Y%m%d").fillna('0').astype(int)
df["dob"] = df["dob"].dt.strftime("%Y%m%d").fillna('0').astype(int)
df["c_jail_in"] = df["c_jail_in"].dt.strftime("%Y%m%d").fillna('0').astype(int)
df["c_jail_out"] = df["c_jail_out"].dt.strftime("%Y%m%d").fillna('0').astype(int)
df["c_offense_date"] = df["c_offense_date"].dt.strftime("%Y%m%d").fillna('0').astype(int)
df["r_offense_date"] = df["r_offense_date"].dt.strftime("%Y%m%d").fillna('0').astype(int)
df["c_arrest_date"] = df["c_arrest_date"].dt.strftime("%Y%m%d").fillna('0').astype(int)
df["vr_offense_date"] = df["vr_offense_date"].dt.strftime("%Y%m%d").fillna('0').astype(int)
df["v_screening_date"] = df["v_screening_date"].dt.strftime("%Y%m%d").fillna('0').astype(int)
df["r_jail_in"] = df["r_jail_in"].dt.strftime("%Y%m%d").fillna('0').astype(int)
df["r_jail_out"] = df["r_jail_out"].dt.strftime("%Y%m%d").fillna('0').astype(int)
df["screening_date"] = df["screening_date"].dt.strftime("%Y%m%d").fillna('0').astype(int)

In [11]:
df.to_csv("datasets/compas/clean.csv", index=False)
df_dirty.to_csv("datasets/compas/dirty.csv", index=False)